In [1]:
#PP1 PCB Detection and Merging

#Import the libraries
import pandas as pd
import numpy as np
import scipy as sp

import os
import sys
import time

import csv
import datetime

import importlib

from matplotlib import pyplot as plt
from matplotlib import dates  as md 
from matplotlib.colors import LogNorm

In [2]:
#Import routines
import routine_rawaccl_to_binryseqTCS2  
import routine_get_pcb_durTCS2          
import routine_elim_spur_pcbTCS2        
import routine_get_pcb_level_dataTCS2   
import routine_merging_algoTCS2         
import routine_merge_pxmty_algoTCS2

Start of definition of the rawaccl_to_binryseq procedure 2020-03-04 18:19:02.661807
Version currently used was compiled at 2019-04-23 08:26
Inputs are inp_vibr_df,downsample_rate,use_filt_accl_flag,use_quant_sig_flag,HYS_LOW_THRESH,HYS_HIGH_THRESH
End   of definition of the procedure 2020-03-04 18:19:02.661991
Start of defining procedure : 2020-03-04 18:19:02.663477
End   of defining procedure : 2020-03-04 18:19:02.663592
Start of defining procedure : 2020-03-04 18:19:02.665980
End   of defining procedure : 2020-03-04 18:19:02.666090
Start of defining procedure : 2020-03-04 18:19:02.669703
Version : 2019-04-23 12:50
End   of defining procedure : 2020-03-04 18:19:02.670156
Start of defining procedure : 2020-03-04 18:19:02.673966
Using version 2019-04-06 08:30
End   of defining procedure : 2020-03-04 18:19:02.674131
Start of defining procedure : 2020-03-04 18:19:02.677130
Version : 2019-04-09 21:59
End   of defining procedure : 2020-03-04 18:19:02.677307


In [3]:
pp2_vibr_file = pd.read_csv('PP2_IMU4.csv',usecols = ['time','ax','ay','az'])
pp2_vibr_file = pp2_vibr_file.sort_values(by="time")
pp2_vibr_file = pp2_vibr_file.reset_index()
pp2_vibr_file = pp2_vibr_file.assign(net_accl = np.sqrt(pp2_vibr_file['ax']**2 + pp2_vibr_file['ay']**2 + pp2_vibr_file['az']**2))

In [4]:
pp2_vibr_file['time']=pd.to_datetime(pp2_vibr_file['time'])
pp2_vibr_file=pp2_vibr_file.rename(columns={'time':'timestamp'})

In [5]:
pp2_vibr_file

,index,timestamp,ax,ay,az,net_accl
0,0,2020-03-02 06:00:00.035768064,-0.012390,0.998779,0.102173,1.004068
1,1,2020-03-02 06:00:00.036829952,-0.012390,0.998779,0.102173,1.004068
2,2,2020-03-02 06:00:00.037848832,-0.006042,0.994934,0.146057,1.005616
3,3,2020-03-02 06:00:00.038857984,-0.006042,0.994934,0.146057,1.005616
4,4,2020-03-02 06:00:00.039846912,-0.006042,0.994934,0.146057,1.005616
5,5,2020-03-02 06:00:00.092904960,-0.006226,1.005127,0.169434,1.019327
6,6,2020-03-02 06:00:00.093966080,-0.006226,1.005127,0.169434,1.019327
7,7,2020-03-02 06:00:00.094968064,-0.003052,1.002075,0.041687,1.002947
8,8,2020-03-02 06:00:00.095980032,-0.003052,1.002075,0.041687,1.002947
9,9,2020-03-02 06:00:00.096983808,-0.003052,1.002075,0.041687,1.002947


In [6]:
filt_sig_pp2 = routine_rawaccl_to_binryseqTCS2.rawaccl_to_binryseq(pp2_vibr_file,1,'Y','Y',0.25,0.8)

START :  2020-03-04 18:19:51.587442
Mean      of normalized signal = -5.1638022525242765e-12
Deviation of normalized signal = 1.0000000000000393
-------------------------------------------------------
Performing Wiener filtering


/home/raghu/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


Performing downsampling by : 1
Performing quantization
Mode = 0.07500000000000001
0 : 0.05 | 6
0.05 : 0.1 | 505026
0.1 : 0.15000000000000002 | 367116
0.15000000000000002 : 0.2 | 0
0.2 : 0.25 | 89624
0.25 : 0.3 | 71979
0.3 : 0.35 | 43850
0.35 : 0.39999999999999997 | 25694
0.39999999999999997 : 0.44999999999999996 | 14476
0.44999999999999996 : 0.49999999999999994 | 11804
0.49999999999999994 : 0.5499999999999999 | 14622
0.5499999999999999 : 0.6 | 17794
0.6 : 0.65 | 18936
0.65 : 0.7000000000000001 | 38786
0.7000000000000001 : 0.7500000000000001 | 20122
0.7500000000000001 : 0.8000000000000002 | 18825
0.8000000000000002 : 0.8500000000000002 | 20090
0.8500000000000002 : 0.9000000000000002 | 21966
0.9000000000000002 : 0.9500000000000003 | 21963
0.9500000000000003 : 1.0000000000000002 | 22893
1.0000000000000002 : 1.0500000000000003 | 23451
1.0500000000000003 : 1.1000000000000003 | 24936
1.1000000000000003 : 1.1500000000000004 | 25691
1.1500000000000004 : 1.2000000000000004 | 25162
1.20000000000

In [7]:
pcb_data_pp2 = routine_get_pcb_durTCS2.get_pcb_dur(filt_sig_pp2.binry_sig)

End of processing at i = 265 j = 266


In [8]:
filt_sig_pp2['cor_binry_sig'] = routine_elim_spur_pcbTCS2.elim_spur_pcb(filt_sig_pp2.binry_sig,pcb_data_pp2,500)

/home/raghu/Desktop/TCS/iiotstream2/streaming2/PP2/routine_elim_spur_pcbTCS2.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cor_binry_sig[STR_IND:END_IND] = 0.0


In [9]:
pcb_data_pp2 = routine_get_pcb_durTCS2.get_pcb_dur(filt_sig_pp2.cor_binry_sig)

End of processing at i = 262 j = 263


In [10]:
pcb_level_pp2 = routine_get_pcb_level_dataTCS2.get_pcb_level_data(filt_sig_pp2,pcb_data_pp2,50)

START :  2020-03-04 18:21:18.101002
str_index = 1565
local_str = 0
end_index = 1564282
local_end = 261
Mode = 2425.0
local_mode = 2425.0
------------------------
Mode calculated at : 2020-03-04 18:21:18.576868
Number of boards detected        = 262
Machine utilization factor       = 35.19206273308685 %
Average   board active duration  = 32.32391857506361 secs
Median    board active duration  = 32.88666666666666 secs
Deviation board active duration  = 9.824141298069273 secs
END   :  2020-03-04 18:21:20.277599
Execution time = 0.03627661666666667 mins


In [11]:
pcb_merge_pp2 = pcb_level_pp2
mrg_binry_pp2 = filt_sig_pp2.cor_binry_sig

THRESHOLD = 0.95
HIGH_END  = 1.50

iter_one = 0
iter_two = 0

In [12]:
pcb_merge_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,1565,2020-03-02 06:00:16.417664000,3302,2020-03-02 06:00:33.487111168,1,17.069447,0.0,0.716289
1,3356,2020-03-02 06:00:34.026341888,4100,2020-03-02 06:00:41.342153984,1,7.315812,0.0,0.306804
2,4428,2020-03-02 06:00:44.556962048,9081,2020-03-02 06:01:30.524560896,1,45.967599,0.0,1.918763
3,9139,2020-03-02 06:01:31.116049920,19070,2020-03-02 06:03:09.556324864,1,98.440275,0.0,4.095258
4,21390,2020-03-02 06:03:32.355293952,24358,2020-03-02 06:04:01.809879040,1,29.454585,0.0,1.223918
5,26156,2020-03-02 06:04:29.543372032,28606,2020-03-02 06:04:53.685988864,1,24.142617,0.0,1.010309
6,31571,2020-03-02 06:05:22.955481088,34027,2020-03-02 06:05:47.265391104,1,24.309910,0.0,1.012784
7,37045,2020-03-02 06:06:17.435219968,39480,2020-03-02 06:06:41.533398016,1,24.098178,0.0,1.004124
8,42542,2020-03-02 06:07:11.694812928,44953,2020-03-02 06:07:35.962265088,1,24.267452,0.0,0.994227
9,47905,2020-03-02 06:08:05.511454976,50346,2020-03-02 06:08:29.763407104,1,24.251952,0.0,1.006598


In [ ]:
PROC_ONE_END_FLAG = 0
while (PROC_ONE_END_FLAG == 0):
    
    iter_one = iter_one + 1
    print('PROC-ONE-ITERATION :', iter_one)
    
    pcb_merge_pp2,mrg_binry_pp2,no_of_merged_pcbs = routine_merge_pxmty_algo2.merge_pxmty_algo    \
    (inp_df        = pcb_merge_pp2                                      \
    ,pxmty_df      = pp2_pxstr_file      \
    ,inp_binry_sig = mrg_binry_pp2                         \
    ,THRESHOLD     = THRESHOLD                                               \
    ,HIGH_END      = HIGH_END                                                \
    )
    
    if (no_of_merged_pcbs == 0):
        PROC_ONE_END_FLAG = 1
    #end-if
    
    print ('=============================================')

In [13]:
PROC_TWO_END_FLAG = 0
while(PROC_TWO_END_FLAG == 0):
    
    iter_two = iter_two + 1
    print('PROC-TWO-ITERATION :', iter_two)
    
    pcb_merge_pp2,mrg_binry_pp2,no_of_merged_pcbs = routine_merging_algoTCS2.merging_algo  \
    (inp_df        = pcb_merge_pp2                                   \
    ,inp_binry_sig = mrg_binry_pp2                                   \
    ,THRESHOLD     = THRESHOLD                                            \
    ,HIGH_END      = HIGH_END                                             \
    ) 
    
    if(no_of_merged_pcbs == 0):
        PROC_TWO_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-TWO-ITERATION : 1
START :  2020-03-04 18:21:33.820004
i = 0 | merge_flag = 1
i = 11 | merge_flag = 1
i = 22 | merge_flag = 1
i = 24 | merge_flag = 1
i = 48 | merge_flag = 1
i = 68 | merge_flag = 1
i = 84 | merge_flag = 1
i = 86 | merge_flag = 1
i = 88 | merge_flag = 1
i = 149 | merge_flag = 1
i = 215 | merge_flag = 1
i = 217 | merge_flag = 1
i = 222 | merge_flag = 1
i = 229 | merge_flag = 1
No. of merged PCBS = 14
END   :  2020-03-04 18:21:34.825097
Execution time = 0.01675155 mins
PROC-TWO-ITERATION : 2
START :  2020-03-04 18:21:34.825653
i = 63 | merge_flag = 1
No. of merged PCBS = 1
END   :  2020-03-04 18:21:35.756024
Execution time = 0.015506183333333333 mins
PROC-TWO-ITERATION : 3
START :  2020-03-04 18:21:35.756344
No. of merged PCBS = 0
END   :  2020-03-04 18:21:36.681661
Execution time = 0.015421949999999999 mins


In [15]:
pcb_merge_pp2

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,1565,2020-03-02 06:00:16.417664000,4100,2020-03-02 06:00:41.342153984,2,24.385259,0.539231,1.023093
1,4428,2020-03-02 06:00:44.556962048,9081,2020-03-02 06:01:30.524560896,1,45.967599,0.000000,1.918763
2,9139,2020-03-02 06:01:31.116049920,19070,2020-03-02 06:03:09.556324864,1,98.440275,0.000000,4.095258
3,21390,2020-03-02 06:03:32.355293952,24358,2020-03-02 06:04:01.809879040,1,29.454585,0.000000,1.223918
4,26156,2020-03-02 06:04:29.543372032,28606,2020-03-02 06:04:53.685988864,1,24.142617,0.000000,1.010309
5,31571,2020-03-02 06:05:22.955481088,34027,2020-03-02 06:05:47.265391104,1,24.309910,0.000000,1.012784
6,37045,2020-03-02 06:06:17.435219968,39480,2020-03-02 06:06:41.533398016,1,24.098178,0.000000,1.004124
7,42542,2020-03-02 06:07:11.694812928,44953,2020-03-02 06:07:35.962265088,1,24.267452,0.000000,0.994227
8,47905,2020-03-02 06:08:05.511454976,50346,2020-03-02 06:08:29.763407104,1,24.251952,0.000000,1.006598
9,53175,2020-03-02 06:08:58.029228032,55599,2020-03-02 06:09:22.505179904,1,24.475952,0.000000,0.999588


In [14]:
filt_sig_pp2['mrg_binry_sig'] = mrg_binry_pp2

In [16]:
pp2EMfile = pd.read_csv('PP2_EM4.csv', usecols=['time', 'c1', 'c2', 'c3'])

In [17]:
raw_pp_current=pp2EMfile

In [18]:
raw_pp_current['time']=pd.to_datetime(raw_pp_current['time'])
raw_pp_current=raw_pp_current.rename(columns={'time':'timestamp'})

In [19]:
raw_pp_current

,timestamp,c1,c2,c3
0,2020-03-02 06:00:00.604685056,0.69,0.75,0.17
1,2020-03-02 06:00:02.444118016,0.67,0.70,0.19
2,2020-03-02 06:00:04.251586816,0.67,0.74,0.16
3,2020-03-02 06:00:06.075225088,0.67,0.72,0.17
4,2020-03-02 06:00:07.898660864,0.67,0.70,0.00
5,2020-03-02 06:00:09.738030848,0.67,0.74,0.19
6,2020-03-02 06:00:11.546132992,0.69,0.73,0.17
7,2020-03-02 06:00:13.352956928,0.71,0.73,0.17
8,2020-03-02 06:00:15.192410880,0.72,0.73,0.25
9,2020-03-02 06:00:16.999851008,1.03,1.76,1.30


In [20]:
raw_pp_current['total_current']=raw_pp_current['c1']+raw_pp_current['c2']+raw_pp_current['c3']

In [23]:
working_times_df=pd.DataFrame()

working_times_df['timestamp']=pcb_merge_pp2['arvl_tmstmp']
working_times_df['working_time']=pcb_merge_pp2['proc_dur']
working_times_df['event']=1

In [24]:
working_times_df

,timestamp,working_time,event
0,2020-03-02 06:00:16.417664000,24.385259,1
1,2020-03-02 06:00:44.556962048,45.967599,1
2,2020-03-02 06:01:31.116049920,98.440275,1
3,2020-03-02 06:03:32.355293952,29.454585,1
4,2020-03-02 06:04:29.543372032,24.142617,1
5,2020-03-02 06:05:22.955481088,24.309910,1
6,2020-03-02 06:06:17.435219968,24.098178,1
7,2020-03-02 06:07:11.694812928,24.267452,1
8,2020-03-02 06:08:05.511454976,24.251952,1
9,2020-03-02 06:08:58.029228032,24.475952,1


In [25]:
working_times_df['energy'] = pcb_merge_pp2.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.dptr_tmstmp) & (x.arvl_tmstmp <= raw_pp_current.timestamp), ['total_current']].sum()*230/3600000, axis=1)

In [26]:
working_times_df

,timestamp,working_time,event,energy
0,2020-03-02 06:00:16.417664000,24.385259,1,0.002690
1,2020-03-02 06:00:44.556962048,45.967599,1,0.004606
2,2020-03-02 06:01:31.116049920,98.440275,1,0.009863
3,2020-03-02 06:03:32.355293952,29.454585,1,0.003092
4,2020-03-02 06:04:29.543372032,24.142617,1,0.002135
5,2020-03-02 06:05:22.955481088,24.309910,1,0.002221
6,2020-03-02 06:06:17.435219968,24.098178,1,0.002421
7,2020-03-02 06:07:11.694812928,24.267452,1,0.002151
8,2020-03-02 06:08:05.511454976,24.251952,1,0.002195
9,2020-03-02 06:08:58.029228032,24.475952,1,0.002246


In [29]:
PP_temp_idle=filt_sig_pp2[['timestamp', 'cor_binry_sig']]
PP_temp_idle['cor_binry_sig']=PP_temp_idle['cor_binry_sig']*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
PP_temp_idle

,timestamp,cor_binry_sig
0,2020-03-02 06:00:01.013089024,-1.0
1,2020-03-02 06:00:01.014142976,-1.0
2,2020-03-02 06:00:01.015149056,-1.0
3,2020-03-02 06:00:01.016155904,-1.0
4,2020-03-02 06:00:01.017158144,-1.0
5,2020-03-02 06:00:01.059937024,-1.0
6,2020-03-02 06:00:01.060986880,-1.0
7,2020-03-02 06:00:01.082187776,-1.0
8,2020-03-02 06:00:01.083239936,-1.0
9,2020-03-02 06:00:01.084258048,-1.0


In [31]:
PP_temp_idle['cor_binry_sig'].iloc[0]=(PP_temp_idle['cor_binry_sig'].iloc[1]+1)%2
PP_temp_idle['cor_binry_sig'].iloc[-1]=(PP_temp_idle['cor_binry_sig'].iloc[-2]+1)%2

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
PP_temp_idle

,timestamp,cor_binry_sig
0,2020-03-02 06:00:01.013089024,0.0
1,2020-03-02 06:00:01.014142976,-1.0
2,2020-03-02 06:00:01.015149056,-1.0
3,2020-03-02 06:00:01.016155904,-1.0
4,2020-03-02 06:00:01.017158144,-1.0
5,2020-03-02 06:00:01.059937024,-1.0
6,2020-03-02 06:00:01.060986880,-1.0
7,2020-03-02 06:00:01.082187776,-1.0
8,2020-03-02 06:00:01.083239936,-1.0
9,2020-03-02 06:00:01.084258048,-1.0


In [33]:
idle_times_raw=sp.signal.find_peaks(PP_temp_idle.cor_binry_sig, width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/72})

In [34]:
idle_times_raw_df

,sample_number,working_time
0,1287,7.708333
1,3328,0.750000
2,4263,4.555556
3,9109,0.805556
4,20229,32.222222
5,25256,24.972222
6,30088,41.180556
7,35535,41.916667
8,41010,42.527778
9,46428,41.000000


In [35]:
for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'start_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'timestamp']=PP_temp_idle.iloc[int(idle_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [36]:
idle_times_raw_df['event']=0

In [37]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [38]:
idle_times_raw_df

,sample_number,working_time,start_time,end_time,timestamp,event,energy
0,1287,7.708333,2020-03-02 06:00:10.962935040,2020-03-02 06:00:16.416654080,2020-03-02 06:00:13.644676864,0,0.000313
1,3328,0.750000,2020-03-02 06:00:33.486105856,2020-03-02 06:00:34.025347840,2020-03-02 06:00:33.772882944,0,0.000000
2,4263,4.555556,2020-03-02 06:00:41.297296896,2020-03-02 06:00:44.555959040,2020-03-02 06:00:42.917140992,0,0.000199
3,9109,0.805556,2020-03-02 06:01:30.523552768,2020-03-02 06:01:31.115037952,2020-03-02 06:01:30.825095936,0,0.000000
4,20229,32.222222,2020-03-02 06:03:09.555331840,2020-03-02 06:03:32.354254848,2020-03-02 06:03:20.955195904,0,0.001125
5,25256,24.972222,2020-03-02 06:04:01.808873984,2020-03-02 06:04:29.542344960,2020-03-02 06:04:20.578515968,0,0.001706
6,30088,41.180556,2020-03-02 06:04:53.684986112,2020-03-02 06:05:22.954466048,2020-03-02 06:05:08.404137984,0,0.001552
7,35535,41.916667,2020-03-02 06:05:47.264384000,2020-03-02 06:06:17.434227968,2020-03-02 06:06:02.414103040,0,0.001848
8,41010,42.527778,2020-03-02 06:06:41.532335104,2020-03-02 06:07:11.693803008,2020-03-02 06:06:56.659739136,0,0.001783
9,46428,41.000000,2020-03-02 06:07:35.906969088,2020-03-02 06:08:05.475575808,2020-03-02 06:07:50.713102080,0,0.001584


In [39]:
working_times_df=working_times_df[['event', 'working_time', 'energy','timestamp']]
idle_times_df=idle_times_raw_df[['event', 'working_time', 'energy', 'timestamp']]

In [40]:
PP_events=pd.DataFrame()

In [41]:
PP_events=PP_events.append(working_times_df)

In [42]:
PP_events=PP_events.append(idle_times_df)

In [43]:
PP_events

,event,working_time,energy,timestamp
0,1,24.385259,0.002690,2020-03-02 06:00:16.417664000
1,1,45.967599,0.004606,2020-03-02 06:00:44.556962048
2,1,98.440275,0.009863,2020-03-02 06:01:31.116049920
3,1,29.454585,0.003092,2020-03-02 06:03:32.355293952
4,1,24.142617,0.002135,2020-03-02 06:04:29.543372032
5,1,24.309910,0.002221,2020-03-02 06:05:22.955481088
6,1,24.098178,0.002421,2020-03-02 06:06:17.435219968
7,1,24.267452,0.002151,2020-03-02 06:07:11.694812928
8,1,24.251952,0.002195,2020-03-02 06:08:05.511454976
9,1,24.475952,0.002246,2020-03-02 06:08:58.029228032
